In [2]:
# 🔧 Встановлення бібліотек (лише якщо ще не встановлено)
!pip install opencv-python pandas tqdm

# 📚 Імпорти
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt

# 📁 Налаштування шляху до зображень
IMAGE_FOLDER = 'images'  # Папка, де зберігаються зображення
OUTPUT_CSV = 'sky_features.csv'

# === Обчислення ознак ===

def compute_cloud_coverage(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cloud_pixels = np.sum(thresh == 255)
    total_pixels = img.shape[0] * img.shape[1]
    return cloud_pixels / total_pixels

def compute_white_pixel_ratio(img):
    white_pixels = np.sum(np.all(img > 240, axis=2))
    total_pixels = img.shape[0] * img.shape[1]
    return white_pixels / total_pixels

def compute_sun_luminance(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray)
    x, y = max_loc
    h, w = img.shape[:2]
    x1, x2 = max(0, x-10), min(w, x+10)
    y1, y2 = max(0, y-10), min(h, y+10)
    roi = gray[y1:y2, x1:x2]
    return np.mean(roi), max_val

def get_time_features(filename):
    try:
        name = os.path.splitext(os.path.basename(filename))[0]
        parts = name.split('_')
        if len(parts) >= 2:
            dt = datetime.strptime(parts[1], '%Y%m%d%H%M')
            return dt.hour, dt.minute
    except:
        pass
    return -1, -1

# === Обробка всіх зображень ===
data = []

for fname in tqdm(sorted(os.listdir(IMAGE_FOLDER))):
    if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(IMAGE_FOLDER, fname)
        img = cv2.imread(path)

        if img is None:
            print(f"Не вдалося прочитати: {fname}")
            continue

        cloud_coverage = compute_cloud_coverage(img)
        white_pixel_ratio = compute_white_pixel_ratio(img)
        sun_lum, sun_max = compute_sun_luminance(img)
        hour, minute = get_time_features(fname)

        data.append({
            'filename': fname,
            'cloud_coverage': cloud_coverage,
            'white_pixel_ratio': white_pixel_ratio,
            'sun_luminance_mean': sun_lum,
            'sun_luminance_max': sun_max,
            'hour': hour,
            'minute': minute
        })

# === Збереження у CSV ===
df = pd.DataFrame(data)
df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Збережено: {OUTPUT_CSV}")
df.head()


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   - -------------------------------------- 1.8/39.5 MB 14.4 MB/s eta 0:00:03
   ------------ --------------------------- 12.1/39.5 MB 34.4 MB/s eta 0:00:01
   -------------- ------------------------- 14.2/39.5 MB 26.2 MB/s eta 0:00:01
   ---------------- ----------------------- 16.0/39.5 MB 21.4 MB/s eta 0:00:02
   ------------------ --------------------- 18.1/39.5 MB 18.4 MB/s eta 0:00:02
   -------------------- ------------------- 19.9/39.5 MB 16.8 MB/s eta 0:00:02
   ---------------------- ----------------- 21.8/39.5 MB 15.6 MB/s eta 0:00:02
   ----------------------- ---------------- 23.6/39.5 MB 14.8 MB/s eta 0:00:02
   ------------------------- -------------- 25.4/39.5 MB 14.1 MB/s eta 0:00:01
   --------------------------- ------------ 27.5/39.5 MB 13.5 MB/s eta 0:00:01
   ----------------------------- ---------- 29.4/39.5 MB 13.1 MB/s eta 0:00:01
   ------------------------------- -------- 31.5/39.5 MB 12.7 


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\Dmitry\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
100%|██████████| 215/215 [00:09<00:00, 23.41it/s]

✅ Збережено: sky_features.csv


,filename,cloud_coverage,white_pixel_ratio,sun_luminance_mean,sun_luminance_max,hour,minute
0,UTC-7_2019_09_08-05_39_36_008433.jpg,0.163999,0.0,38.74,40.0,-1,-1
1,UTC-7_2019_09_08-05_39_46_009511.jpg,0.143443,0.0,39.84,41.0,-1,-1
2,UTC-7_2019_09_08-05_39_56_010590.jpg,0.150242,0.0,39.58,41.0,-1,-1
3,UTC-7_2019_09_08-05_40_06_012085.jpg,0.155692,0.0,40.76,43.0,-1,-1
4,UTC-7_2019_09_08-05_40_16_013181.jpg,0.156292,0.0,40.84,43.0,-1,-1
